# DisALEXI Single Image Example

In [1]:
import ee
from IPython.display import Image
from importlib import reload  # Python 3 only
# from imp import reload      # Python 2/3 via futures modules

from disalexi import disalexi
disalexi = reload(disalexi)

ee.Initialize()

ndvi_palette = ['#EFE7E1', '#003300']
ta_palette = ['#ff0000', '#ffff00', '#00ffff', '#0000ff']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

# Input Landsat 8 Image

In [2]:
# Note, DisALEXI image must be in 2015 due to ancillary data availability 
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_043033_20150805')
Image(url=landsat_img.select([3, 2, 1]).getThumbURL({'min': 0.0, 'max': 0.3}))

In [3]:
#
image_region = landsat_img.geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
image_crs = landsat_img.select(['B2']).projection().crs().getInfo()
image_transform = landsat_img.select(['B2']).projection().getInfo()['transform']
coarse_transform = image_transform[:]
coarse_transform[0] = 240
coarse_transform[4] = 240

In [4]:
# Build the DisALEXI object
disalexi_obj = disalexi.DisALEXI.fromLandsatTOA(landsat_img)

## Compute ALEXI scale air temperature (aggregated LST for now)

In [5]:
# Projecting the Ta image to the Landsat image for display purposes
ta_img = disalexi_obj.ta() \
    .reproject(crs=image_crs, crsTransform=coarse_transform)
Image(url=ta_img.getThumbURL({
    'min': 280, 'max': 320, 'region': image_region, 'palette': ','.join(ta_palette)}))

## Compute Landsat scale ET (1.25 * NDVI for now)

In [6]:
# Skip Ta smoothing for now
# Why is Ta an input to the function?
et_img = disalexi_obj.et(ta_img)
Image(url=et_img.getThumbURL({
    'min': 0, 'max': 1.2, 'palette': ','.join(et_palette)}))